In [1]:
import sys
import os
import pandas as pd
import numpy as np
from Bio import SeqIO
import re

In [2]:
dataset = 'JC2019'

print('Loading raw data for', dataset, '...')
data = pd.read_csv('/Users/maryamkoddus/Documents/maryam-ko-QMUL-MSc-Project/01_input_data/raw_data/142561_0_supp_250799_pj97x9.csv', header=0)
print('Raw data loaded.')
data

Loading raw data for JC2019 ...
Raw data loaded.


,ppSTY ID,Charge,Amino acid,Position in peptide,Sequence window,Localization score,Localization probability,Phospho (STY) Probabilities,Proteins,Positions within proteins,...,Gene names,Raw intensity (A) TiOx.Testis(sample.URO0059),Raw intensity (B) TiOx.Testis(sample.URO0126),Raw intensity (C) TiOx.Testis(sample.URO0158),Normalised intensity (A) TiOx.Testis(sample.URO0059),Normalised intensity (B) TiOx.Testis(sample.URO0126),Normalised intensity (C) TiOx.Testis(sample.URO0158),Spectral count (A) TiOx.Testis(sample.URO0059),Spectral count (B) TiOx.Testis(sample.URO0126),Spectral count (C) TiOx.Testis(sample.URO0158)
0,1330,3,S,14,PKKLEKEEEEGISQESSEEEQ__________,55.158,1.000000,KLEKEEEEGIS(1)QES(1)S(1)EEEQ,P17096;P17096-2,102;91,...,HMGA1,4.620000e+08,3.190000e+08,2.300000e+08,2.750000e+08,1.580000e+08,1.280000e+08,22,22,18
1,1331,3,S,15,KKLEKEEEEGISQESSEEEQ___________,55.158,1.000000,KLEKEEEEGIS(1)QES(1)S(1)EEEQ,P17096;P17096-2,103;92,...,HMGA1,9.890000e+06,0.000000e+00,3.210000e+06,5.890000e+06,0.000000e+00,1.790000e+06,19,16,13
2,1555,2,S,10,EDRKPKAEEDEILNRSPRNRKPRRE______,294.930,1.000000,AEEDEILNRS(1)PR,P27824-3;P27824;P27824-2,475;583;618,...,CANX,8.600000e+09,5.390000e+09,3.880000e+09,5.120000e+09,2.660000e+09,2.170000e+09,7,7,7
3,1032,3,S,4,AYRKLLEGEEERLRLSPSPTSQRSRGRASSH,148.240,1.000000,LRLS(1)PS(1)PTSQR,P02545;P02545-3;P02545-5;P02545-4;P02545-6;P02...,390;390;291;278;390;390,...,LMNA,1.670000e+09,3.230000e+09,2.650000e+09,9.950000e+08,1.600000e+09,1.480000e+09,13,11,13
4,2270,3,S,3,GWRAAQDRDQIYRRRSPSPYYSRGGYRSRSR;GYDRYEDYDYRYRR...,122.580,1.000000,RRS(1)PS(1)PYYSR,P62995;P62995-3;Q13595;Q13595-3;Q13595-4,264;164;260;159;158,...,TRA2B;TRA2A,5.430000e+07,7.410000e+07,6.590000e+07,3.230000e+07,3.670000e+07,3.690000e+07,15,15,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7936,4577,3,S,5,LRLPRTGRLLRAKRPSSEDFVFL________,43.030,0.500000,AKRPS(0.5)S(0.5)EDFVFL,Q86TL0-2;Q86TL0,134;467,...,ATG4D,0.000000e+00,1.380000e+06,0.000000e+00,0.000000e+00,6.850000e+05,0.000000e+00,0,1,0
7937,4578,3,S,6,RLPRTGRLLRAKRPSSEDFVFL_________,43.030,0.500000,AKRPS(0.5)S(0.5)EDFVFL,Q86TL0-2;Q86TL0,135;468,...,ATG4D,0.000000e+00,1.380000e+06,0.000000e+00,0.000000e+00,6.850000e+05,0.000000e+00,0,1,0
7938,3701,3,S,3,RPKWNSWDPRRQRQLSMSSADSADAKRTREE,54.898,0.951810,QLS(0.952)MS(0.038)S(0.01)ADSADAKR,Q5JSZ5-5;Q5JSZ5,416;416,...,PRRC2B,0.000000e+00,0.000000e+00,1.270000e+06,0.000000e+00,0.000000e+00,7.070000e+05,0,0,1
7939,5685,2,S,3,__________MIRAFSFPVSPERGRLRGWLE,40.880,0.998613,AFS(0.999)FPVS(0.001)PER,Q96B18,6,...,DACT3,1.230000e+06,0.000000e+00,0.000000e+00,7.320000e+05,0.000000e+00,0.000000e+00,1,0,0


In [3]:
# filter data to keep only those with localization probability >= 0.85
data = data[data['Localization probability'] >= 0.85] 

# Filtering out semi-colons from 'Amino acid', 'Positions within proteins', and 'Gene names' columns
data = data[~data['Amino acid'].str.contains(';', na=False)]
data = data[~data['Positions within proteins'].str.contains(';', na=False)]
data = data[~data['Gene names'].str.contains(';', na=False)]
data

,ppSTY ID,Charge,Amino acid,Position in peptide,Sequence window,Localization score,Localization probability,Phospho (STY) Probabilities,Proteins,Positions within proteins,...,Gene names,Raw intensity (A) TiOx.Testis(sample.URO0059),Raw intensity (B) TiOx.Testis(sample.URO0126),Raw intensity (C) TiOx.Testis(sample.URO0158),Normalised intensity (A) TiOx.Testis(sample.URO0059),Normalised intensity (B) TiOx.Testis(sample.URO0126),Normalised intensity (C) TiOx.Testis(sample.URO0158),Spectral count (A) TiOx.Testis(sample.URO0059),Spectral count (B) TiOx.Testis(sample.URO0126),Spectral count (C) TiOx.Testis(sample.URO0158)
8,1484,2,S,2,_____________MASGVAVSDGVIKVFNDM,128.710,1.000000,AS(1)GVAVSDGVIK,P23528,3,...,CFL1,2.250000e+09,6.200000e+09,5.360000e+09,1.340000e+09,3.070000e+09,3.000000e+09,3,9,8
14,2842,2,S,3,YAITVYRLDDSDREGSVKSFSLGPLRKAVTL,41.695,1.000000,EGS(1)VKS(1)FS(1)LGPLR,Q13325,195,...,IFIT5,0.000000e+00,0.000000e+00,1.130000e+10,0.000000e+00,0.000000e+00,6.340000e+09,0,0,1
15,2843,2,S,6,TVYRLDDSDREGSVKSFSLGPLRKAVTLNPD,41.695,1.000000,EGS(1)VKS(1)FS(1)LGPLR,Q13325,198,...,IFIT5,0.000000e+00,0.000000e+00,1.130000e+10,0.000000e+00,0.000000e+00,6.340000e+09,0,0,1
16,2844,2,S,8,YRLDDSDREGSVKSFSLGPLRKAVTLNPDNS,41.695,1.000000,EGS(1)VKS(1)FS(1)LGPLR,Q13325,200,...,IFIT5,0.000000e+00,0.000000e+00,1.130000e+10,0.000000e+00,0.000000e+00,6.340000e+09,0,0,1
19,5645,4,S,26,SELGPEQLEAEVGESSDEEPVESRAQRLRRT,151.480,0.999031,APEPLGPADQSELGPEQLEAEVGES(1)S(0.999)DEEPVES(0....,Q969G5,166,...,PRKCDBP,8.880000e+08,2.490000e+09,1.460000e+09,5.290000e+08,1.230000e+09,8.190000e+08,10,12,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7919,7970,3,S,16,KGDVEGSQSQDEGEGSGESERGSGSQSSVPS,47.730,0.899693,KGDVEGSQSQDEGEGS(0.9)GES(0.1)ER,Q9UQE7,1074,...,SMC3,0.000000e+00,0.000000e+00,1.910000e+06,0.000000e+00,0.000000e+00,1.070000e+06,0,0,1
7930,4639,2,S,3,PGPASGSTFRGRRDVSGSWERDQQVEAAQRA,44.612,0.984060,DVS(0.984)GS(0.016)WER,Q86VR2,26,...,FAM134C,1.630000e+06,0.000000e+00,0.000000e+00,9.710000e+05,0.000000e+00,0.000000e+00,1,0,0
7931,2081,3,S,3,SAYNMANSFTFTRSLSRYESLLRTASSPDSL,60.157,0.986372,S(0.011)LS(0.986)RY(0.002)ES(0.001)LLR,P51805,1596,...,PLXNA3,0.000000e+00,1.610000e+06,0.000000e+00,0.000000e+00,7.970000e+05,0.000000e+00,0,1,0
7932,7697,2,S,1,QQQVHSSSVNAAARRSVFESNSDNWFLGRDE,62.940,0.999927,S(1)VFESNSDNWFLGR,Q9ULL0,1387,...,KIAA1210,1.520000e+06,0.000000e+00,0.000000e+00,9.060000e+05,0.000000e+00,0.000000e+00,1,0,0


In [4]:
# filter data
data['Sequence window'] = data['Sequence window'].str.replace('_', '')
data

,ppSTY ID,Charge,Amino acid,Position in peptide,Sequence window,Localization score,Localization probability,Phospho (STY) Probabilities,Proteins,Positions within proteins,...,Gene names,Raw intensity (A) TiOx.Testis(sample.URO0059),Raw intensity (B) TiOx.Testis(sample.URO0126),Raw intensity (C) TiOx.Testis(sample.URO0158),Normalised intensity (A) TiOx.Testis(sample.URO0059),Normalised intensity (B) TiOx.Testis(sample.URO0126),Normalised intensity (C) TiOx.Testis(sample.URO0158),Spectral count (A) TiOx.Testis(sample.URO0059),Spectral count (B) TiOx.Testis(sample.URO0126),Spectral count (C) TiOx.Testis(sample.URO0158)
8,1484,2,S,2,MASGVAVSDGVIKVFNDM,128.710,1.000000,AS(1)GVAVSDGVIK,P23528,3,...,CFL1,2.250000e+09,6.200000e+09,5.360000e+09,1.340000e+09,3.070000e+09,3.000000e+09,3,9,8
14,2842,2,S,3,YAITVYRLDDSDREGSVKSFSLGPLRKAVTL,41.695,1.000000,EGS(1)VKS(1)FS(1)LGPLR,Q13325,195,...,IFIT5,0.000000e+00,0.000000e+00,1.130000e+10,0.000000e+00,0.000000e+00,6.340000e+09,0,0,1
15,2843,2,S,6,TVYRLDDSDREGSVKSFSLGPLRKAVTLNPD,41.695,1.000000,EGS(1)VKS(1)FS(1)LGPLR,Q13325,198,...,IFIT5,0.000000e+00,0.000000e+00,1.130000e+10,0.000000e+00,0.000000e+00,6.340000e+09,0,0,1
16,2844,2,S,8,YRLDDSDREGSVKSFSLGPLRKAVTLNPDNS,41.695,1.000000,EGS(1)VKS(1)FS(1)LGPLR,Q13325,200,...,IFIT5,0.000000e+00,0.000000e+00,1.130000e+10,0.000000e+00,0.000000e+00,6.340000e+09,0,0,1
19,5645,4,S,26,SELGPEQLEAEVGESSDEEPVESRAQRLRRT,151.480,0.999031,APEPLGPADQSELGPEQLEAEVGES(1)S(0.999)DEEPVES(0....,Q969G5,166,...,PRKCDBP,8.880000e+08,2.490000e+09,1.460000e+09,5.290000e+08,1.230000e+09,8.190000e+08,10,12,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7919,7970,3,S,16,KGDVEGSQSQDEGEGSGESERGSGSQSSVPS,47.730,0.899693,KGDVEGSQSQDEGEGS(0.9)GES(0.1)ER,Q9UQE7,1074,...,SMC3,0.000000e+00,0.000000e+00,1.910000e+06,0.000000e+00,0.000000e+00,1.070000e+06,0,0,1
7930,4639,2,S,3,PGPASGSTFRGRRDVSGSWERDQQVEAAQRA,44.612,0.984060,DVS(0.984)GS(0.016)WER,Q86VR2,26,...,FAM134C,1.630000e+06,0.000000e+00,0.000000e+00,9.710000e+05,0.000000e+00,0.000000e+00,1,0,0
7931,2081,3,S,3,SAYNMANSFTFTRSLSRYESLLRTASSPDSL,60.157,0.986372,S(0.011)LS(0.986)RY(0.002)ES(0.001)LLR,P51805,1596,...,PLXNA3,0.000000e+00,1.610000e+06,0.000000e+00,0.000000e+00,7.970000e+05,0.000000e+00,0,1,0
7932,7697,2,S,1,QQQVHSSSVNAAARRSVFESNSDNWFLGRDE,62.940,0.999927,S(1)VFESNSDNWFLGR,Q9ULL0,1387,...,KIAA1210,1.520000e+06,0.000000e+00,0.000000e+00,9.060000e+05,0.000000e+00,0.000000e+00,1,0,0


In [5]:
def match_seq_to_genename(dataset, seq_column):
    '''
    Maps amino acid sequences to gene names using the loaded fasta file.
    
    args:
    =====
    dataset: <pd.Dataframe> with a column of amino acid sequences
    seq_column: <str> column name containing amino acid sequences
    
    out:
    ====
    dataset: <pd.Dataframe> with an additional column containing gene names
    '''    

    fasta_sequence = list(SeqIO.parse(open(f'/Users/maryamkoddus/Documents/maryam-ko-QMUL-MSc-Project/01_input_data/raw_data/UP000005640_9606.fasta'), "fasta"))
    
    
    gene_dict = {}
    
    # iterate over rows in seq_column
    for i in dataset[seq_column]:
        print(i)
        i_str = str(i)
        for seq_record in fasta_sequence:
            matches = re.findall(i_str, str(seq_record.seq))
            if matches:
                print(f"Match found for sequence: {seq_record}")
                gene_name_match = re.search(r"GN=(\w+)", seq_record.description)
                print('Gene name match:', gene_name_match)
                # gene_name_match = re.search("GN=(\w+)", seq_record.description)
                if gene_name_match:
                    gene_name = gene_name_match.group(1)
                    gene_dict[i] = gene_name
                    print(f"Match found: {i_str} -> {gene_name}")
                else: 
                    print(f"No gene name found in description for sequence: {i_str}")
    
    # map sequences to gene names           
    dataset['GeneName'] = dataset[seq_column].map(gene_dict) 
    print('Amino acid sequences matched to gene names.')
    return dataset 

In [6]:
data = match_seq_to_genename(data, 'Sequence window')

MASGVAVSDGVIKVFNDM
Match found for sequence: ID: sp|P23528|COF1_HUMAN
Name: sp|P23528|COF1_HUMAN
Description: sp|P23528|COF1_HUMAN Cofilin-1 OS=Homo sapiens OX=9606 GN=CFL1 PE=1 SV=3
Number of features: 0
Seq('MASGVAVSDGVIKVFNDMKVRKSSTPEEVKKRKKAVLFCLSEDKKNIILEEGKE...KPL')
Gene name match: <re.Match object; span=(55, 62), match='GN=CFL1'>
Match found: MASGVAVSDGVIKVFNDM -> CFL1
YAITVYRLDDSDREGSVKSFSLGPLRKAVTL
Match found for sequence: ID: sp|Q13325|IFIT5_HUMAN
Name: sp|Q13325|IFIT5_HUMAN
Description: sp|Q13325|IFIT5_HUMAN Interferon-induced protein with tetratricopeptide repeats 5 OS=Homo sapiens OX=9606 GN=IFIT5 PE=1 SV=1
Number of features: 0
Seq('MSEIRKDTLKAILLELECHFTWNLLKEDIDLFEVEDTIGQQLEFLTTKSRLALY...LSI')
Gene name match: <re.Match object; span=(106, 114), match='GN=IFIT5'>
Match found: YAITVYRLDDSDREGSVKSFSLGPLRKAVTL -> IFIT5
TVYRLDDSDREGSVKSFSLGPLRKAVTLNPD
Match found for sequence: ID: sp|Q13325|IFIT5_HUMAN
Name: sp|Q13325|IFIT5_HUMAN
Description: sp|Q13325|IFIT5_HUMAN Interfero

In [7]:
data['Phosphosite'] = data['Amino acid'].astype(str) + '(' + data['Positions within proteins'].astype(str) + ')'

In [8]:
# Keep only 'Phosphosite' and ratio columns
keepcols = ['Phosphosite'] + ['GeneName'] + [col for col in data.columns if 'Normalised intensity' in col]
data = data[keepcols]
data

,Phosphosite,GeneName,Normalised intensity (A) TiOx.Testis(sample.URO0059),Normalised intensity (B) TiOx.Testis(sample.URO0126),Normalised intensity (C) TiOx.Testis(sample.URO0158)
8,S(3),CFL1,1.340000e+09,3.070000e+09,3.000000e+09
14,S(195),IFIT5,0.000000e+00,0.000000e+00,6.340000e+09
15,S(198),IFIT5,0.000000e+00,0.000000e+00,6.340000e+09
16,S(200),IFIT5,0.000000e+00,0.000000e+00,6.340000e+09
19,S(166),CAVIN3,5.290000e+08,1.230000e+09,8.190000e+08
...,...,...,...,...,...
7919,S(1074),SMC3,0.000000e+00,0.000000e+00,1.070000e+06
7930,S(26),RETREG3,9.710000e+05,0.000000e+00,0.000000e+00
7931,S(1596),PLXNA3,0.000000e+00,7.970000e+05,0.000000e+00
7932,S(1387),KIAA1210,9.060000e+05,0.000000e+00,0.000000e+00


In [9]:
Intensity_columns = [col for col in data.columns if 'Normalised intensity' in col]
data[Intensity_columns] = data[Intensity_columns].apply(pd.to_numeric, errors='coerce')

/var/folders/1v/y_z33m9d25d2tszlfrvnp2300000gn/T/ipykernel_4255/1662973474.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[Intensity_columns] = data[Intensity_columns].apply(pd.to_numeric, errors='coerce')


In [10]:
def log2_transform(dataset):
    '''
    Log2 transform a dataset.
    
    args:
    =====
    dataset: <pd.Dataframe>
    
    out:
    ====
    dataset: <pd.Dataframe> with log2 transformed values

    '''
    cols_to_transform = dataset[Intensity_columns]
    dataset[Intensity_columns] = cols_to_transform.apply(np.log2)
    print('Data has been log2 transformed.')
    return dataset

In [11]:
data = log2_transform(data)
print(f"DataFrame after log2 transformation:\n{data}") # Print the DataFrame after log2 transformation
data

/var/folders/1v/y_z33m9d25d2tszlfrvnp2300000gn/T/ipykernel_4255/3201699711.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset[Intensity_columns] = cols_to_transform.apply(np.log2)


Data has been log2 transformed.
DataFrame after log2 transformation:
     Phosphosite  GeneName  \
8           S(3)      CFL1   
14        S(195)     IFIT5   
15        S(198)     IFIT5   
16        S(200)     IFIT5   
19        S(166)    CAVIN3   
...          ...       ...   
7919     S(1074)      SMC3   
7930       S(26)   RETREG3   
7931     S(1596)    PLXNA3   
7932     S(1387)  KIAA1210   
7939        S(6)     DACT3   

      Normalised intensity (A) TiOx.Testis(sample.URO0059)  \
8                                             30.319586      
14                                                 -inf      
15                                                 -inf      
16                                                 -inf      
19                                            28.978692      
...                                                 ...      
7919                                               -inf      
7930                                          19.889112      
7931        

,Phosphosite,GeneName,Normalised intensity (A) TiOx.Testis(sample.URO0059),Normalised intensity (B) TiOx.Testis(sample.URO0126),Normalised intensity (C) TiOx.Testis(sample.URO0158)
8,S(3),CFL1,30.319586,31.515592,31.482315
14,S(195),IFIT5,-inf,-inf,32.561836
15,S(198),IFIT5,-inf,-inf,32.561836
16,S(200),IFIT5,-inf,-inf,32.561836
19,S(166),CAVIN3,28.978692,30.196011,29.609288
...,...,...,...,...,...
7919,S(1074),SMC3,-inf,-inf,20.029179
7930,S(26),RETREG3,19.889112,-inf,-inf
7931,S(1596),PLXNA3,-inf,19.604220,-inf
7932,S(1387),KIAA1210,19.789152,-inf,-inf


In [12]:
def create_phos_ID(dataset):
    '''
    Concatenates GeneName and Phosphosite columns.
    
    args:
    =====
    dataset: <pd.Dataframe> with columns 'GeneName' and 'Phosphosite'
    
    out:
    ====
    dataset: <pd.Dataframe> with 'phosphosite_ID' column and 'GeneName' + 'Phosphosite' columns dropped
    '''
    dataset.loc[:, 'phosphosite_ID'] = dataset['GeneName'].astype(str) + '_' + dataset['Phosphosite'].astype(str)
    dataset = dataset.drop(columns=['Phosphosite', 'GeneName'])
    print('Phosphosite IDs created.')
    return dataset

data = create_phos_ID(data) # call function to create phosphosite_ID column

print('Phosphosite IDs created.')

/var/folders/1v/y_z33m9d25d2tszlfrvnp2300000gn/T/ipykernel_4255/3553972964.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset.loc[:, 'phosphosite_ID'] = dataset['GeneName'].astype(str) + '_' + dataset['Phosphosite'].astype(str)


Phosphosite IDs created.
Phosphosite IDs created.


In [13]:
def clean_phosID_col(data):
    data = data[~data.phosphosite_ID.str.contains('nan', case = False)]
    data = data[~data.phosphosite_ID.str.contains(';', case = False)] # remove rows containing ';' in phosphosite_ID column
    data = data[~data.phosphosite_ID.str.contains('-', case = False)] # remove rows containing '-' in phosphosite_ID column
    
    # check whether there are any phosphosites with multiple measurements
    data_grouped = data.groupby(by = 'phosphosite_ID')
    if len(data) != len(data_grouped):
        data = data_grouped.mean()
        data.reset_index(inplace=True) # reset index
        print('Phosphosites with multiple measurements have been averaged')
    else:
        print('There are no phosphosites with multiple measurements')
        
    print(data)
        
    data = data.replace([np.inf, -np.inf], np.nan)
        
    if data.columns[0] != 'phosphosite_ID':
        phosphosite_ID = data.pop('phosphosite_ID')
        data.insert(0, 'phosphosite_ID', phosphosite_ID)
    return data

In [14]:
data = clean_phosID_col(data)
print("After cleaning phosphosite_ID column:")
data

Phosphosites with multiple measurements have been averaged
      phosphosite_ID  Normalised intensity (A) TiOx.Testis(sample.URO0059)  \
0        AATF_S(316)                                          24.885765      
1        AATF_S(320)                                          24.885765      
2        AATF_S(321)                                          24.885765      
3       ABCF1_S(228)                                          24.101494      
4     ABHD17B_S(282)                                          20.697103      
...              ...                                                ...      
1970     ZNRF2_S(19)                                          23.697103      
1971     ZNRF2_S(82)                                          22.972461      
1972       ZYX_S(15)                                          22.675730      
1973   ZZEF1_S(1537)                                          21.353802      
1974   ZZEF1_S(1540)                                          21.353802      

    

,phosphosite_ID,Normalised intensity (A) TiOx.Testis(sample.URO0059),Normalised intensity (B) TiOx.Testis(sample.URO0126),Normalised intensity (C) TiOx.Testis(sample.URO0158)
0,AATF_S(316),24.885765,25.060852,24.809313
1,AATF_S(320),24.885765,25.060852,24.809313
2,AATF_S(321),24.885765,25.060852,24.809313
3,ABCF1_S(228),24.101494,24.187069,23.759388
4,ABHD17B_S(282),20.697103,21.713977,20.880169
...,...,...,...,...
1970,ZNRF2_S(19),23.697103,24.609640,24.231692
1971,ZNRF2_S(82),22.972461,24.101494,NaN
1972,ZYX_S(15),22.675730,NaN,NaN
1973,ZZEF1_S(1537),21.353802,21.482469,21.082128


In [15]:
data.to_csv(f'/Users/maryamkoddus/Documents/maryam-ko-QMUL-MSc-Project/01_input_data/PreprocessedDatasets/JC2019.csv', index=False)


print(dataset, 'has been saved to CSV successfully!', data)

JC2019 has been saved to CSV successfully!       phosphosite_ID  Normalised intensity (A) TiOx.Testis(sample.URO0059)  \
0        AATF_S(316)                                          24.885765      
1        AATF_S(320)                                          24.885765      
2        AATF_S(321)                                          24.885765      
3       ABCF1_S(228)                                          24.101494      
4     ABHD17B_S(282)                                          20.697103      
...              ...                                                ...      
1970     ZNRF2_S(19)                                          23.697103      
1971     ZNRF2_S(82)                                          22.972461      
1972       ZYX_S(15)                                          22.675730      
1973   ZZEF1_S(1537)                                          21.353802      
1974   ZZEF1_S(1540)                                          21.353802      

      Normalised int